# Step-by-step implementation
The following code demonstrates how to integrate step-back prompting into an RAG pipeline. Let’s break it down step-by-step:
1. Import necessary libraries
2. Set up the OpenAI API key
3. Few-shot learning for step-back prompting
4. Build the step-back prompt
5. Retrieve information
6. Build the RAG chain

## 1. Import necessary libraries

In [0]:
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain import hub

## 2. Set up the OpenAI API key

In [0]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] = ""  # Add your OpenAI API key
if OPENAI_API_KEY == "":
    raise ValueError("Please set the OPENAI_API_KEY environment variable")

## 3. Few-shot learning for step-back prompting

In [0]:
# Few Shot Examples
examples = [
    {
        "input": "Did the Beatles ever write a book?",
        "output": "What types of creative works did the Beatles produce?"
    },
    {
        "input": "Was Albert Einstein a musician?",
        "output": "What fields did Albert Einstein work in?"
    }
]

# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

## 4. Build the step-back prompt

In [0]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        few_shot_prompt,
        ("user", "{question}"),
    ]
)

In [0]:
question_gen = prompt | ChatOpenAI(temperature=0) | StrOutputParser()

In [0]:
question = "Did Leonardo da Vinci invent the printing press?"

In [0]:
question_gen.invoke({"question": question})

## 5. Retrieve information

In [0]:
search = DuckDuckGoSearchAPIWrapper(max_results=4)

def retriever(query):
    return search.run(query)

In [0]:
retriever(question)

In [0]:
retriever(question_gen.invoke({"question": question}))

## 6. Build the RAG chain

In [0]:
response_prompt = hub.pull("langchain-ai/stepback-answer")

In [0]:
chain = (
    {
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        "step_back_context": question_gen | retriever,
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

In [0]:
chain.invoke({"question": question})